In [1]:
#read data

import pandas as pd
train_file_path = "project_2_train/project_2_train/dara 1_train.csv"
train_data = pd.read_csv(train_file_path)
train_data.head()

,example_id,text,aspect_term,term_location,class
0,2333_1,Obviously one of the most important features o...,human interface,69--84,0
1,1805_1,Good for every day computing and web browsing.,every day computing,9--28,1
2,2782_2,while the keyboard itself is alright[comma] th...,mouse command buttons,115--136,-1
3,1385_0,Again[comma] the same problem[comma] the right...,right speaker,29--42,-1
4,1423_0,My problem was with DELL Customer Service.,DELL Customer Service,20--41,-1


In [2]:
class_value = train_data[' class'].value_counts(dropna=False)
class_value.describe()

count      3.000000
mean     734.333333
std      264.258081
min      436.000000
25%      632.000000
50%      828.000000
75%      883.500000
max      939.000000
Name:  class, dtype: float64

In [3]:
class_value.head()

 1    939
-1    828
 0    436
Name:  class, dtype: int64

In [4]:
X = train_data.drop([" class","example_id"," text"],axis=1)
X.head()

,aspect_term,term_location
0,human interface,69--84
1,every day computing,9--28
2,mouse command buttons,115--136
3,right speaker,29--42
4,DELL Customer Service,20--41


In [5]:
X.shape[0]

2203

In [6]:
Y = train_data[" class"]
Y.head()

0    0
1    1
2   -1
3   -1
4   -1
Name:  class, dtype: int64

In [7]:
#training bigram tagger

from nltk.corpus import brown

brown_tagged_sents = brown.tagged_sents()
size = int(len(brown_tagged_sents) * 0.9)

train_sents = brown_tagged_sents[:size]
test_sents = brown_tagged_sents[size:]

In [8]:
import nltk
def build_backoff_tagger(train_sents):
    t0 = nltk.DefaultTagger('NN')
    t1 = nltk.UnigramTagger(train_sents, backoff=t0)
    t2 = nltk.BigramTagger(train_sents, backoff=t1)
    return t2

In [9]:
ngram_tagger = build_backoff_tagger(train_sents)

In [10]:
ngram_tagger.evaluate(test_sents)

0.9125751765470128

In [11]:
text_data = train_data[" text"]
new_text=[]
for idx,text_record in enumerate(text_data):
    aspect_loc = train_data.loc[idx," term_location"]
    locations = aspect_loc.split("--")
    current_text = text_data[idx].replace("[comma]", ",")
    #current_text = current_text[:int(locations[0])] + "<aspect>" + current_text[int(locations[1]):]
    new_text.append([current_text])

In [12]:

new_text_aspect=[]
for idx,text_record in enumerate(new_text):
    aspect_loc = train_data.loc[idx," term_location"]
    locations = aspect_loc.split("--")
    current_text = text_record[0]
    current_text = current_text[:int(locations[0])] + "<aspect>" + current_text[int(locations[1]):]
    new_text_aspect.append(current_text)

In [13]:
#extracting words in context 0f 5 from target term
import nltk
from nltk import word_tokenize
from nltk.util import ngrams

sent_list=[]

remove_list = [",",".",".)","..)",").","?","!"]
    
    
for idx,text_word in enumerate(new_text_aspect):
    locations = train_data.loc[idx," term_location"].split("--")
    sent = text_word
    sent_left = sent[:int(locations[0])]
    #token_left = nltk.word_tokenize(sent_left)
    #print(token_left)
    #bigram_left = nltk.bigrams(token_left)
    #bigram_left = *map(' '.join, bigrm)
    sent_right = sent[int(locations[0])+8:]
    #print(sent_right)
    
    #token_right = nltk.word_tokenize(sent_right)
    if sent_right not in remove_list:
        sent_surf = ' '.join(sent_left.split()[-5:])+' '+' '.join(sent_right.split()[:5])
    else:
        sent_surf = ' '.join(sent_left.split()[-5:])
    #print(sent_surf)
    sent_list.append([sent_surf])
    #bigrm_right = [b for b in nltk.bigrams(token_right)]
    #bigrm_right = *map(' '.join, bigrm)
    #print(bigrm_right)
    #sent_left_words = ngrams(token_left,2)
    #sent_right_words = ngrams(sent_right,1)
    #words = sent_left_words[-3:] + sent_right_words[:3]
    #context_word.append(words)
    #print(sent_left_words)
#print(context_word)
#print(sent_list)

In [14]:
import nltk
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer, WordPunctTokenizer
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.collocations import *
from nltk.metrics import BigramAssocMeasures

features=[]

stops = set(stopwords.words('english'))
for line in sent_list:
    #tokenizer = WordPunctTokenizer()
    tokens = nltk.word_tokenize(line[0])
    #stemmer = PorterStemmer()
    bigram_finder = BigramCollocationFinder.from_words(tokens)
    bigrams = bigram_finder.nbest(BigramAssocMeasures.chi_sq, 500)
    #bigrams = ngrams(line,2)
    for bigram_tuple in bigrams:
        x = "%s %s" % bigram_tuple
        tokens.append(x)
    
    #result = [b for b in nltk.bigrams(line) if b[0] not in stops and b[1] not in stops]

    result = [' '.join([w for w in x.split()]) for x in tokens if x.lower() not in stops and len(x) > 8]
    features.append(result)

In [15]:
feature_main=[]
for f in features:
    if f:
        feature_main.append(f)
        
        
vocab_list=[]
for f in features:
    for ele in f:
        vocab_list.append(ele)
        
        


vocab_list_set = set(vocab_list)

In [16]:
features_uni=[]
stop_list = stopwords.words('english') +['.',',',')','?','!','(','-',':','The','I','It','We',"'",'They','My','But','us']

stops = set(stop_list)
for line in sent_list:
    tokenizer = WordPunctTokenizer()
    tokens = nltk.word_tokenize(line[0])
    #print(tokens)
    #stemmer = PorterStemmer()
    #bigram_finder = BigramCollocationFinder.from_words(tokens)
    #bigrams = bigram_finder.nbest(BigramAssocMeasures.chi_sq, 500)
    #bigrams = ngrams(line,2)
    #for bigram_tuple in bigrams:
    #    x = "%s %s" % bigram_tuple
    #    tokens.append(x)
    
    #result = [b for b in nltk.bigrams(line) if b[0] not in stops and b[1] not in stops]
    new_token = []
    for token in tokens:
        if token not in stops:
            new_token.append(token)
    features_uni.append(new_token)
    #if tokens not in stops:
    #    features_uni.append(result)

In [17]:
vocab_list_uni=[]
for f in features_uni:
    for ele in f:
        vocab_list_uni.append(ele)
        
        


vocab_list_set_uni = set(vocab_list_uni)

In [18]:
#features_uni
#vocab_list_uni

In [19]:
text_df = pd.DataFrame({'text':new_text_aspect})
#print (text_df)

In [20]:
#uni_df = pd.DataFrame(columns =vocab_list_set_uni )
import numpy as np
uni_df = pd.DataFrame(0, index=np.arange(X.shape[0]), columns =vocab_list_set_uni)


In [21]:
uni_df.head()

,badly,exposed,secures,Oh,render,Adjust,quiet,Cut,last,frustrating,...,review,save,effeciency,2003,sweet,Unless,AMD,applications,load,similar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
#bag of words
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(vocabulary=vocab_list_set)
X_train_bag = count_vect.fit_transform(text_df["text"])
X_term_freq = pd.DataFrame(X_train_bag.toarray())

In [23]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_bag)
#X_train_tfidf.shape
X_tfidf = pd.DataFrame(X_train_tfidf.toarray())

In [24]:
#dropping columns with sum less than 25
cols_with_less_than_25 = [col for col in X_term_freq.columns 
                                 if abs(X_term_freq[col].sum()) < 1]
reduced_X_train = X_term_freq.drop(cols_with_less_than_25, axis=1)
#reduced_X_val  = val_X.drop(cols_with_missing, axis=1)
#print("Mean Absolute Error from dropping columns with Missing Values:")
#print(getScore(reduced_X_train, train_Y,reduced_X_val,  val_Y))

In [25]:
X_tfidf[900].sum()

0.0

In [26]:
cols_with_less_than_25[900]

912

In [27]:
reduced_X_train.shape

(2203, 331)

In [28]:
#dropping columns with stop words
stop_list = stopwords.words('english')
cols_with_stop_words = [col for col in reduced_X_train.columns 
                                 if reduced_X_train.columns in stop_list ]
reduced_X_train_without_stops = reduced_X_train.drop(cols_with_stop_words, axis=1)

/Applications/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/base.py:3726: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = op(self.values, np.asarray(other))


In [29]:
reduced_X_train_without_stops.shape

(2203, 331)

In [30]:
import nltk
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer, WordPunctTokenizer
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.collocations import *
from nltk.metrics import BigramAssocMeasures
features_uni_tagged=[]
#stop_list = stopwords.words('english') +['.',',',')','?','!','(','-',':','The','I','It','We',"'",'They','My','But','us']

#stops = set(stop_list)
for line in sent_list:
    tokenizer = WordPunctTokenizer()
    tokens = nltk.word_tokenize(line[0])
    #print(tokens)
    #stemmer = PorterStemmer()
    #bigram_finder = BigramCollocationFinder.from_words(tokens)
    #bigrams = bigram_finder.nbest(BigramAssocMeasures.chi_sq, 500)
    #bigrams = ngrams(line,2)
    #for bigram_tuple in bigrams:
    #    x = "%s %s" % bigram_tuple
    #    tokens.append(x)
    tagged_tokens = ngram_tagger.tag(tokens)
    #result = [b for b in nltk.bigrams(line) if b[0] not in stops and b[1] not in stops]
    #new_token = []
    #for token in tokens:
        #if token not in stops:
     #       new_token.append(token)
    features_uni_tagged.append(tagged_tokens)

In [31]:
new_tagged_Sent=[]
for feature in features_uni_tagged:
    sent = ""
    for tuples in feature:
        sent = sent +" " +tuples[0]+"_"+tuples[1]
    
    new_tagged_Sent.append(sent.strip())

In [32]:
text_df_tagged = pd.DataFrame({'text_tagged':new_tagged_Sent})

In [33]:
text_df_tagged.head()

,text_tagged
0,any_DTI computer_NN is_BEZ the_AT ``_``
1,Good_JJ for_IN and_CC web_NN browsing_VBG ._.
2,hollow_JJ sound_NN when_WRB using_VBG the_AT
3,"Again_RB ,_, the_AT same_AP problem_NN ,_, the..."
4,My_PP$ problem_NN was_BEDZ with_IN


In [34]:
#bag of words with pos tage
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_bag_tagged = count_vect.fit_transform(text_df_tagged["text_tagged"])
X_term_freq_tagged = pd.DataFrame(X_train_bag_tagged.toarray(),columns=count_vect.get_feature_names())

In [35]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_bag_tagged)
#X_train_tfidf.shape
X_tfidf_tagged = pd.DataFrame(X_train_tfidf.toarray(),columns=count_vect.get_feature_names())

In [36]:
#dropping columns with sum less than 2
cols_with_less_than_25 = [col for col in X_tfidf_tagged.columns 
                                 if abs(X_tfidf_tagged[col].sum()) < 2]
reduced_X_train_tagged = X_tfidf_tagged.drop(cols_with_less_than_25, axis=1)

In [37]:
reduced_X_train_tagged.shape

(2203, 539)

In [38]:
reduced_X_train_tagged.head()

,10_cd,1_cd,2_cd,3_cd,4_cd,7_cd,__nn,_in,_nn,a_at,...,works_vbz,worse_jjr,worst_jjt,worth_jj,would_md,xp_nn,year_nn,you_ppo,you_ppss,your_pp
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
from nltk.tokenize import word_tokenize as WordTokenizer

def word_tokenizer(data, col):
    token=[]
    for item in data[col]:
         token.append(WordTokenizer(item))

    return token


In [40]:
word_token = word_tokenizer(text_df, "text")
#df.insert(index, 'token_column', token)
#print(token)
#tokenize_data = pd.DataFrame(token)
#word_token

In [41]:
#tokenize_data.head()
#word_token

In [42]:
def penn_to_wn(tag):
    
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [43]:
import nltk
nltk.download('sentiwordnet')
nltk.download('stopwords')

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/sylvesterraj/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sylvesterraj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
word_token[39]

['The',
 'processor',
 'is',
 'very',
 'quick',
 'and',
 'effective',
 'as',
 'I',
 'load',
 '<',
 'aspect',
 '>',
 'and',
 'applications',
 '.']

In [45]:
stop_words=['aspect','<','>','is','the','of','for','and','.',',']
word_token_reduced=[]
for token in word_token:
    new_token =  [x for x in token if x not in stop_words]
    word_token_reduced.append(new_token)

In [46]:
word_token_reduced[39]

['The',
 'processor',
 'very',
 'quick',
 'effective',
 'as',
 'I',
 'load',
 'applications']

In [47]:
#polarity shifter
negWords = ['no', 'not', 'never', "n't"]
def polarityShifter(word, index, negList):
    if(word in negWords):
        negList.append(index + 1)
        negList.append(index + 2)
    
    return negList

In [48]:
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
lemmatizer = WordNetLemmatizer()

def sentiWeights(words):
    tagged_word = []
    tagged = pos_tag(words)
    #print(tagged)
    #sentiment = 0.0
    idx=-1
    for word, tag in tagged:
        sentiment = 0.0
        negIndex = []
        prevIndex = -1
        idx = idx + 1 
        wn_tag = penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV,wn.VERB):
            continue

        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        if not lemma:
            continue

        synsets = wn.synsets(lemma, pos=wn_tag)
        if not synsets:
            continue

        # Take the first sense, the most common
        synset = synsets[0]
        swn_synset = swn.senti_synset(synset.name())
        negIndex = polarityShifter(word, idx, negIndex)
        sentiment += swn_synset.pos_score() - swn_synset.neg_score()
        if(idx in negIndex and prevIndex != idx - 1):
            prevIndex = idx
            sentiment *= -1
        tagged_word.append([word,sentiment,idx])
        
    return tagged_word

In [49]:
sent_score_word = [] 
for token in word_token_reduced:
    #print(token)
    sent_score_word.append(sentiWeights(token))
#print(sent_score_word)

In [50]:
#tagged_word
sent_score_word[0]

[['Obviously', 0.5, 0],
 ['most', 0.0, 2],
 ['important', 0.875, 3],
 ['features', 0.0, 4],
 ['computer', 0.0, 6]]

In [51]:
len(sent_score_word)

2203

In [52]:
#word_token_reduced
def getScoreForSentiments(feature_columns_sentiword,sent_score_word):
    my_df= []
    for senti in sent_score_word:
        number_of_pos = 0
        number_of_neg = 0
        sum_of_scores = 0
        maximum_token_score = 0
        minimum_token_score = 0
        for se in senti:
            if se[1] > 0:
                number_of_pos = number_of_pos+1

            if se[1] < 0:
                number_of_neg = number_of_neg+1

            sum_of_scores = sum_of_scores + se[1]

            if se[1] > maximum_token_score:
                maximum_token_score = se[1]

            if se[1] < minimum_token_score:
                minimum_token_score = se[1]

        #print(number_of_pos)

        d = {
            feature_columns_sentiword[0] : number_of_pos,  
            feature_columns_sentiword[1] : number_of_neg,
            feature_columns_sentiword[2] : sum_of_scores,
            feature_columns_sentiword[3] : maximum_token_score,
            feature_columns_sentiword[4] : minimum_token_score
        }
        my_df.append(d)

    my_df = pd.DataFrame(my_df)
    return my_df

In [53]:
#my_df_senti.shape

NameError: name 'my_df_senti' is not defined

In [59]:
#bing liu's opinion lexicon - negative words
f = open('opinion-lexicon-English/negative-words.txt', 'r',encoding = "ISO-8859-1")
negative_words_liu = f.read().splitlines()
f.close()

In [60]:
f = open('opinion-lexicon-English/positive-words.txt', 'r',encoding = "ISO-8859-1")
positive_words_liu = f.read().splitlines()
f.close()

In [61]:
#negative_words_liu

In [62]:
def sentiWeightsLiu(words):
    tagged_word = []
    tagged = pos_tag(words)
    #print(tagged)
    #sentiment = 0.0
    idx=-1
    for word, tag in tagged:
        sentiment = 0.0
        negIndex = []
        prevIndex = -1
        idx = idx + 1
        wn_tag = penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV,wn.VERB):
            continue

        if word in positive_words_liu:
            sentiment = 1.0
        if word in negative_words_liu:
            sentiment = -1.0
            
        negIndex = polarityShifter(word, idx, negIndex)

        if(idx in negIndex and prevIndex != idx - 1):
            prevIndex = idx
            sentiment *= -1
            
        tagged_word.append([word,sentiment,idx])
        
    return tagged_word

In [63]:
#assigning weights on basis of liu's opinion lexicon

sent_score_word_liu = [] 
for token in word_token_reduced:
    #print(token)
    sent_score_word_liu.append(sentiWeightsLiu(token))

In [64]:
sent_score_word_liu[1]

[['Good', 0.0, 0], ['web', 0.0, 1], ['browsing', 0.0, 2]]

In [65]:
aspect_locations_tokens = []
for word_ in word_token:
    aspect_locations_tokens.append(word_.index("aspect")-2)

In [66]:
sent_score_word_liu_new = []
for idx,sent_liu in enumerate(sent_score_word_liu):
    start_range = aspect_locations_tokens[idx]-3
    end_range = aspect_locations_tokens[idx]+3
    sent_new =[]
    for sent in sent_liu:
        if sent[2] >= start_range and sent[2] <= end_range:
            sent[1] = sent[1] * 2
        sent_new.append(sent)
    sent_score_word_liu_new.append(sent_new)

In [67]:
sent_score_word_new = []
for idx,sent_senti in enumerate(sent_score_word):
    start_range = aspect_locations_tokens[idx]-3
    end_range = aspect_locations_tokens[idx]+3
    sent_new =[]
    for sent in sent_senti:
        if sent[2] >= start_range and sent[2] <= end_range:
            sent[1] = sent[1] * 2
        sent_new.append(sent)
    sent_score_word_new.append(sent_new)

In [68]:
sent_score_word_new[0]

[['Obviously', 0.5, 0],
 ['most', 0.0, 2],
 ['important', 0.875, 3],
 ['features', 0.0, 4],
 ['computer', 0.0, 6]]

In [69]:
sent_score_word_liu_new[2]

[['keyboard', 0.0, 1],
 ['alright', 0.0, 3],
 ['plate', 0.0, 4],
 ['cheap', -1.0, 7],
 ['plastic', 0.0, 8],
 ['makes', 0.0, 9],
 ['hollow', -1.0, 11],
 ['sound', 0.0, 12],
 ['using', 0.0, 14]]

In [70]:
sent_score_word_liu[2]

[['keyboard', 0.0, 1],
 ['alright', 0.0, 3],
 ['plate', 0.0, 4],
 ['cheap', -1.0, 7],
 ['plastic', 0.0, 8],
 ['makes', 0.0, 9],
 ['hollow', -1.0, 11],
 ['sound', 0.0, 12],
 ['using', 0.0, 14]]

In [71]:
#mpqa lexicon
f = open('mpqa/mpqa.txt', 'r',encoding = "ISO-8859-1")
mpqa_lines = f.read().splitlines()
f.close()

FileNotFoundError: [Errno 2] No such file or directory: 'mpqa/mpqa.txt'

In [72]:
mpqa_sent_score_word = []
for mpqa in mpqa_lines:
    mpqas = mpqa.split(" ")
    mpqa_types = mpqas[0].split("=")
    mpqa_type = mpqa_types[1]
    mpqa_words = mpqas[2].split("=")
    mpqa_word = mpqa_words[1]
    #print(mpqas)
    #print(mpqas[5])
    mpqa_polarities = mpqas[5].split("=")
    mpqa_polarity = mpqa_polarities[1]
    score = 0.0
    if mpqa_polarity == "negative":
        if mpqa_type == "weaksubj":
            score = -0.5
        else:
            score = -1.0
            
    if mpqa_polarity == "positive":
        if mpqa_type == "weaksubj":
            score = 0.5
        else:
            score = 1.0
    
    mpqa_sent_score_word.append([mpqa_word,score])
    

NameError: name 'mpqa_lines' is not defined

In [73]:
mpqa_sent_score_word[4]

IndexError: list index out of range

In [74]:
for mpqa_w in mpqa_sent_score_word:
    if 'be' == mpqa_w[0]:
        print(mpqa_w[1])
        break

In [75]:
def sentiWeightsMpqa(words):
    tagged_word = []
    tagged = pos_tag(words)
    #print(tagged)
    
    idx=-1
    for word, tag in tagged:
        sentiment = 0.0
        negIndex = []
        prevIndex = -1
        idx = idx + 1
        wn_tag = penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV,wn.VERB):
            continue

        for mpqa_w in mpqa_sent_score_word:
            if word == mpqa_w[0]:
                sentiment = mpqa_w[1]
                break
        
        negIndex = polarityShifter(word, idx, negIndex)

        if(idx in negIndex and prevIndex != idx - 1):
            prevIndex = idx
            sentiment *= -1
            
        tagged_word.append([word,sentiment,idx])
        
    return tagged_word

In [76]:
sent_score_word_mpqa = [] 
for token in word_token_reduced:
    #print(token)
    sent_score_word_mpqa.append(sentiWeightsMpqa(token))

In [77]:
sent_score_word_mpqa[4]

[['problem', 0.0, 1], ['was', 0.0, 2]]

In [78]:
# populating unigram dataframe with weighted sentiment scores of mpqa

for idx,sent_word_list in enumerate(sent_score_word_mpqa):
    for sent_word in sent_word_list:
        if sent_word[0] in uni_df.columns:
            uni_df.loc[idx,sent_word[0]] = sent_word[1]
            
    

In [79]:
# populating unigram dataframe with weighted sentiment scores of liu

for idx,sent_word_list in enumerate(sent_score_word_liu):
    for sent_word in sent_word_list:
        if sent_word[0] in uni_df.columns:
            uni_df.loc[idx,sent_word[0]] = uni_df.loc[idx,sent_word[0]] + sent_word[1]

In [80]:
# populating unigram dataframe with weighted sentiment scores of sentiword

for idx,sent_word_list in enumerate(sent_score_word_new):
    for sent_word in sent_word_list:
        if sent_word[0] in uni_df.columns:
            uni_df.loc[idx,sent_word[0]] = (uni_df.loc[idx,sent_word[0]] + sent_word[1])

In [81]:
uni_df.shape

(2203, 2350)

In [82]:
uni_df = uni_df.add_suffix('_without_tag')

In [83]:
reduced_uni_df = reduced_uni_df.add_suffix('_without_tag')

NameError: name 'reduced_uni_df' is not defined

In [ ]:
#Amazon unigram lexicon

f = open('opinion-lexicon-English/Amazon-laptops-electronics-reviews-unigrams.txt', 'r',encoding = "ISO-8859-1")
domain_specific_lex = f.read().splitlines()
f.close()

In [ ]:
#Yelp unigram lexicon
f = open('opinion-lexicon-English/Yelp-restaurant-reviews-unigrams.txt', 'r',encoding = "ISO-8859-1")
domain_specific_lex = f.read().splitlines()
f.close()

In [ ]:
lex_score_list = {}
lex_list = []
for lexes in domain_specific_lex:
        splitted_lex = lexes.split(',')
        lex_score_list[splitted_lex[0]] = splitted_lex[1]
        lex_list.append(splitted_lex[0])

In [ ]:
def sentiWeightsLex(words):
    tagged_word = []
    #tagged = pos_tag(words)
    #print(tagged)
    idx=-1
   
    prevIndex = -1
    
    #for word, tag in tagged:
        #wn_tag = penn_to_wn(tag)
        #if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV,wn.VERB):
            #continue
    for word in words:
        sentiment = 0.0
        negIndex = []
        idx = idx + 1
        if word in lex_list:
            sentiment = float(lex_score_list[word])
        
        negIndex = polarityShifter(word, idx, negIndex)

        if(idx in negIndex and prevIndex != idx - 1):
            prevIndex = idx
            sentiment *= -1
            #print(word, sentiment, idx)
            
        tagged_word.append([word,sentiment,idx])
        
    return tagged_word

In [ ]:
senti_score_lex = [] 
for token in word_token_reduced:
    senti_score_lex.append(sentiWeightsLex(token))

In [ ]:
senti_score_lex[0]

In [ ]:
#Bit of modification

def getScoreForSentimentsLex(feature_columns_sentiword,sent_score_word):
    my_df= []
    for senti in sent_score_word:
        total_pos = 0
        total_neg = 0
        sum_of_scores = 0
        maximum_token_score = 0
        minimum_token_score = 0
        for se in senti:
            if se[1] > 0:
                total_pos += se[1]

            if se[1] < 0:
                total_neg += se[1]

            sum_of_scores = sum_of_scores + se[1]

            if se[1] > maximum_token_score:
                maximum_token_score = se[1]

            if se[1] < minimum_token_score:
                minimum_token_score = se[1]

        #print(number_of_pos)

        d = {
            feature_columns_sentiword[0] : total_pos,  
            feature_columns_sentiword[1] : total_neg,
            feature_columns_sentiword[2] : sum_of_scores,
            feature_columns_sentiword[3] : maximum_token_score,
            feature_columns_sentiword[4] : minimum_token_score
        }
        my_df.append(d)

    my_df = pd.DataFrame(my_df)
    return my_df

In [ ]:
# populating unigram dataframe with weighted sentiment scores of sentiword

for idx,sent_word_list in enumerate(senti_score_lex):
    for sent_word in sent_word_list:
        if sent_word[0] in uni_df.columns:
            uni_df.loc[idx,sent_word[0]] = (uni_df.loc[idx,sent_word[0]] + sent_word[1])/4

In [ ]:
uni_df.shape

In [ ]:
#dropping columns with absolute sum less than 0.001
cols_with_less_than_abs_sum = [col for col in uni_df.columns 
                                 if abs(uni_df[col].sum()) < 2]
reduced_uni_df = uni_df.drop(cols_with_less_than_abs_sum, axis=1)

In [ ]:
reduced_uni_df.shape

In [ ]:
feature_columns_sentiword_lex = ['total positives lex','total negatives lex','sum of scores lex','maximum token score lex','minimum token score lex']


my_df_lex = getScoreForSentimentsLex(feature_columns_sentiword_lex,senti_score_lex)

my_df_lex.head()

In [ ]:
feature_columns_sentiword = ['number of positives','number of negatives','sum of scores','maximum token score','minimum token score']

my_df_senti = getScoreForSentiments(feature_columns_sentiword,sent_score_word_new)

In [54]:
feature_columns_sentiword_liu = ['number of positives liu','number of negatives liu','sum of scores liu','maximum token score liu','minimum token score liu']


my_df_liu = getScoreForSentiments(feature_columns_sentiword_liu,sent_score_word_liu_new)

NameError: name 'sent_score_word_liu_new' is not defined

In [55]:
feature_columns_sentiword_mpqa = ['number of positives mpqa','number of negatives mpqa','sum of scores mpqa','maximum token score mpqa','minimum token score mpqa']


my_df_mpqa = getScoreForSentiments(feature_columns_sentiword_mpqa,sent_score_word_mpqa)

NameError: name 'sent_score_word_mpqa' is not defined

In [56]:
my_df_liu.head()

NameError: name 'my_df_liu' is not defined

In [84]:
vocab_list_senti = []


In [85]:
train_data.loc[1," term_location"]

'9--28'

In [86]:
#train_data[" text"]

In [87]:
#new_text_aspect

In [88]:
len(features)

2203

In [89]:
len(feature_main) 

2063

In [90]:
len(vocab_list_set)

5752

In [91]:
#vocab_list_set

In [92]:
#vocab_list_set

In [93]:
#cv = CountVectorizer(vocabulary=vocab_list_set)
#cv.fit_transform(train_data[" text"]).toarray()
#cv.vocabulary_

In [94]:
len(vocab_list)

8507

In [95]:
X_term_freq.shape

(2203, 5752)

In [96]:
X_term_freq.head()

,0,1,2,3,4,5,6,7,8,9,...,5742,5743,5744,5745,5746,5747,5748,5749,5750,5751
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
reduced_X_train.shape

(2203, 331)

In [98]:
#for idx,ngrm in enumerate(ngram_set):
#    print('ngram: {0}\n'.format(ngrm))
#    print('ngram.shape: {0}'.format(ngrm.shape))
#    print('vectorizer.vocabulary_: {0}'.format(vectorizers[idx].vocabulary_))

In [99]:
train_data_mix =[]
train_data_mix = my_df_liu.join(my_df_senti)

NameError: name 'my_df_liu' is not defined

In [100]:
#train_data_mix.shape

In [101]:
train_data_mix = reduced_X_train.join(my_df_senti)

NameError: name 'my_df_senti' is not defined

In [102]:
train_data_mix = train_data_mix.join(my_df_liu)

AttributeError: 'list' object has no attribute 'join'

In [103]:
train_data_mix = train_data_mix.join(my_df_mpqa)

AttributeError: 'list' object has no attribute 'join'

In [104]:
reduced_uni_df.shape

NameError: name 'reduced_uni_df' is not defined

In [105]:

train_data_mix = train_data_mix.join(reduced_uni_df)

AttributeError: 'list' object has no attribute 'join'

In [106]:
train_data_mix = train_data_mix.join(my_df_lex)

AttributeError: 'list' object has no attribute 'join'

In [107]:
train_data_mix.shape

AttributeError: 'list' object has no attribute 'shape'

In [108]:
train_data_mix = train_data_mix.join(reduced_X_train_tagged)

AttributeError: 'list' object has no attribute 'join'

In [109]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_Y, val_Y = train_test_split(train_data_mix, Y,random_state = 0)
example_ids = val_X['example_id']

ValueError: Found input variables with inconsistent numbers of samples: [0, 2203]

In [611]:
#Random Forest train
from sklearn.ensemble import RandomForestClassifier
forest_model = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=30, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=683, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
forest_model.fit(train_X,train_Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=30, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=683, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [612]:
#Random Forest predict
from sklearn.metrics import accuracy_score
forest_pred = forest_model.predict(val_X)
accuracy_score(val_Y, forest_pred)

0.7259528130671506

In [613]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
#Random Forest using cross validation and confusion matrix
forest_model = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=30, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=683, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
predicted = cross_val_predict(forest_model, train_data_mix, Y, cv=10)
conf_mat_forest = confusion_matrix(Y,predicted,labels=[-1,0,1])
#calclulateF1(conf_mat_forest)
#print(calclulateF1(conf_mat_naive))
print(precision_recall_fscore_support(Y,predicted))
print("Overal Accuracy")
print(accuracy_score(Y, predicted))

(array([0.69186603, 0.71084337, 0.78729839]), array([0.87318841, 0.2706422 , 0.83173589]), array([0.77202349, 0.39202658, 0.8089073 ]), array([828, 436, 939]))
Overal Accuracy
0.7362687244666364


In [338]:
from xgboost import XGBClassifier


xgboost = XGBClassifier(learning_rate= 0.2, n_estimators= 300,max_depth=30)
xgboost.fit(train_X,train_Y)
xgboost_pred = xgboost.predict(val_X)
accuracy_score(val_Y, xgboost_pred)

KeyboardInterrupt: 

In [614]:
from sklearn.ensemble import BaggingClassifier 
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
clf1 = BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=99999,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=99, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)
clf2 = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=683, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
clf3 = MLPClassifier()

vote_model = VotingClassifier(estimators=[('bag', clf1), ('rf', clf2), ('mlp', clf3)], voting='hard')
vote_model.fit(train_X,train_Y)
vote_pred = vote_model.predict(val_X)
accuracy_score(val_Y, vote_pred)
def genOutput("Noman_Mulla_Sylvester_Raj_Data-1.txt",example_ids,vote_pred)

/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7332123411978222

In [463]:
clf1 = BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=99999,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=49, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)
clf2 = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=683, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
clf3 = MLPClassifier()
vote_model = VotingClassifier(estimators=[('bag', clf1), ('rf', clf2), ('mlp', clf3)], voting='hard')
predicted = cross_val_predict(vote_model, train_data_mix, Y, cv=10)
conf_mat_forest = confusion_matrix(Y,predicted,labels=[-1,0,1])
#calclulateF1(conf_mat_forest)
#print(calclulateF1(conf_mat_naive))
print(precision_recall_fscore_support(Y,predicted))
print("Overal Accuracy")
print(accuracy_score(Y, predicted))

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

(array([0.7110885 , 0.62992126, 0.78674948]), array([0.8442029 , 0.36697248, 0.80937167]), array([0.7719492 , 0.46376812, 0.79790026]), array([828, 436, 939]))
Overal Accuracy
0.7349069450748978


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [73]:
# svm grid search hyper parameter tuning

from sklearn import svm, grid_search
from sklearn.model_selection import GridSearchCV
def svc_param_selection(X, y, nfolds):
    # 
    Cs = [0.01,0.1,1.5, 3]
    #
    gammas = [0.001, 0.003,0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    #grid_search.best_params_
    #means = clf.cv_results_['mean_test_score']
    #stds = clf.cv_results_['std_test_score']
    
    print("Best Params")
    print(grid_search.best_params_)
    return grid_search.best_params_

In [74]:
svc_param_selection(train_data_mix, Y,5)

Best Params
{'C': 0.01, 'gamma': 0.001}


{'C': 0.01, 'gamma': 0.001}

In [615]:
#Import Library
from sklearn import svm
#Assumed you have, X (predictor) and Y (target) for training data set and x_test(predictor) of test_dataset
# Create SVM classification object 
# C = [3.1,3.5,3.55,3.981,3.764,3.333]
# gamma = [0.01,0.03,0.05,0.055,0.066,0.077,0.091,0.092,0.0933,0.1]
model = svm.SVC(kernel='rbf', C=3, gamma=0.1) 
# there is various option associated with it, like changing kernel, gamma and C value. Will discuss more # about it in next section.Train the model using the training sets and check score
model.fit(train_X, train_Y)
print(model.score(train_X, train_Y))
#Predict Output
predicted= model.predict(val_X)
accuracy_score(val_Y, predicted)

0.9309927360774818


0.6950998185117967

In [263]:
svm_model = svm.SVC(kernel='rbf', C=3.5, gamma=0.01) 
predicted = cross_val_predict(svm_model, train_data_mix, Y, cv=10)
conf_mat_forest = confusion_matrix(Y,predicted,labels=[-1,0,1])
#calclulateF1(conf_mat_forest)
#print(calclulateF1(conf_mat_naive))
print(precision_recall_fscore_support(Y,predicted))
print("Overal Accuracy")
print(accuracy_score(Y, predicted))

(array([0.55503876, 0.34291581, 0.76032389]), array([0.4447205 , 0.26382306, 0.86783734]), array([0.4937931 , 0.29821429, 0.81053086]), array([ 805,  633, 2164]))
Overal Accuracy
0.667129372570794


In [ ]:
def genOutput(output_file, example_ids, predicted):
    for i in range(0, len(example_ids)):
        out_file.write(example_ids[i] ";;"+ predicted[i])